In [2]:
using DataFrames
using CSV
using JSON

In [68]:
using PowerSystems
using NamedTuples
using TimeSeries

In [5]:
function add_date_time(df)
    df[:DateTime] = collect(DateTime(df[1,:Year],df[1,:Month],df[1,:Day],Int(df[1,:Period])-1):Hour(1):
                        DateTime(df[end,:Year],df[end,:Month],df[end,:Day],Int(df[end,:Period])-1))
    delete!(df, [:Year,:Month,:Day,:Period])
    
end

add_date_time (generic function with 1 method)

### TODO :
- change the order of the columns
- make datetime index for the data frame

In [37]:
function make_forecast_dict(df,resolution::Base.Dates.Period,horizon::Int,devices::Array{PowerSystems.Generator,1})
    Forecasts = Dict{String,Any}()
    Forecasts["Deterministic"] = Dict{String,Any}()
    time_delta = Minute(df[2,:DateTime]-df[1,:DateTime])
    initialtime = df[1,:DateTime]
    last_date = df[end,:DateTime]
    ts_dict = Dict{Any,Dict{Int,TimeSeries.TimeArray}}()
    device_names = [d.name for d in devices]
    for name in convert(Array{String},names(df))
        if name in device_names
            ts_raw = TimeSeries.TimeArray(df[:,:DateTime],df[:,Symbol(name)])
            for ts in initialtime:resolution:last_date
                ts_dict[ts] = Dict{Int,TimeSeries.TimeArray}(1 => ts_raw[ts:time_delta:(ts+resolution)])
            end
            device = [d for d in devices if d.name ==name ] 
            Forecasts["Deterministic"][name]=Dict{String,Any}("horizon" =>horizon,
                                                                    "resolution" => resolution,
                                                                    "interval" => time_delta,
                                                                    "initialtime" => initialtime,
                                                                    "device" => device[1],
                                                                    "data" => ts_dict
                                                                        )
        end
    end
    return Forecasts
end

make_forecast_dict (generic function with 3 methods)

### TODO:
- Add Checks
- gen_names array arg ?

In [7]:
function make_forecast_array(dict)
    Forecasts =Array{PowerSystems.Forecast}(0)
            for (time_key,time_dict) in dict
                push!(Forecasts,Deterministic(time_dict["device"],time_dict["horizon"],
                                time_dict["resolution"],time_dict["interval"],
                                time_dict["initialtime"],
                                time_dict["data"] 
                                ))
            end
    return Forecasts
end

make_forecast_array (generic function with 1 method)

## Run Test
- TODO : Read Json to Dict

- Read Csv data from RTS-GMLC Hydro DAY_AHEAD

In [43]:
HydroTS_raw = CSV.read("../timeseries_data_files/HYDRO/DAY_AHEAD_hydro.csv");

- Add DateTime

In [44]:
add_date_time(HydroTS_raw);
gen_names =convert(Array{String},names(HydroTS_raw)[1:end-1]);

- Make Dict

- Write Json

In [ ]:
for (type_key,type_fc) in Forecasts_dict
    for (key,det_dicts) in type_fc
        stringdata =JSON.json(det_dicts, 3)
        open("JSON/$key.json", "w") do f
            write(f, stringdata)
         end
    end
end

- Make Array

In [ ]:
Forecast = make_forecast_array(Forecasts);

- Example data Extraction

In [ ]:
Forecast_back[1].device_name,  Forecast_back[1].initialtime, Forecast_back[1].resolution

In [ ]:
Forecasts["Deterministic"]["215_HYDRO_3"]["data"][DateTime(2020,02,13,00,00,00)][1]

In [30]:
bus_raw = readtable("../SourceData/bus.csv");
gen_raw = readtable("../SourceData/gen.csv");
HydroTS_raw = readtable("HYDRO/DAY_AHEAD_hydro.csv")
PVTS_raw = readtable("PV/DAY_AHEAD_pv.csv")
RTPVTS_raw = readtable("RTPV/DAY_AHEAD_rtpv.csv")
WindTS_raw = readtable("WIND/DAY_AHEAD_wind.csv")
CSPTS_raw  =  CSV.read("../timeseries_data_files/CSP/DAY_AHEAD_Natural_Inflow.csv");

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] #readtable#229(::Bool, ::Char, ::Array{Char,1}, ::Char, ::Array{String,1}, ::Array{String,1}, ::Array{String,1}, ::Bool, ::Int64, ::Array{Symbol,1}, ::Array{Any,1}, ::Bool, ::Char, ::Bool, ::Int64, ::Array{Int64,1}, ::Bool, ::Symbol, ::Bool, ::Bool, ::DataFrames.#readtable, ::String) at /Users/sdalvi/.julia/v0.6/DataFrames/src/deprecated.jl:1045
 [3] readtable(::String) at /Users/sdalvi/.julia/v0.6/DataFrames/src/deprecated.jl:1045
 [4] include_string(::String, ::String) at ./loading.jl:522
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/sdalvi/.julia/v0.6/IJulia/src/execute_request.jl:158
 [6] (::Compat.#inner#18{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/sdalvi/.julia/v0.6/Compat/src/Compat.jl:378
 [7] eventloop(::ZMQ.Socket) at /Users/sdalvi/.julia/v0.6/IJulia/src/eventloop.jl:8
 [8] (::IJulia.##14#17)() at ./task.jl:335
while loading In[30], in expression starting

In [14]:
bus = Array{Bus}(0)
for i in 1:nrow(bus_raw)
    push!(bus,PowerSystems.Bus(bus_raw[i,1],bus_raw[i,2],bus_raw[i,4],bus_raw[i,8],bus_raw[i,7],
            @NT(min=0.95,max=1.05),bus_raw[i,3]))
end

In [32]:
Generators =Array{PowerSystems.Generator}(0)
DayAhead  = collect(DateTime("1/1/2024  00:00:00", "d/m/y  H:M:S"):Hour(1):DateTime("31/12/2024  23:00:00", "d/m/y  H:M:S"))
for gen in 1:nrow(gen_raw)
    if gen_raw[gen,:Fuel] in ["Oil","Coal","NG","Nuclear"]
        varcost=gen_raw[1,[:Output_pct_0,:Output_pct_1,:Output_pct_2,:Output_pct_3,:Output_pct_4,:HR_avg_0,:HR_incr_1,:HR_incr_2,:HR_incr_3,:HR_incr_4]]
        fuel_cost = gen_raw[gen,:Fuel_Price_MMBTU]
        start_hr =gen_raw[gen,:Start_Heat_Warm_MBTU]
        bus_id = [bus[i] for i in 1:length(bus) if bus[i].number == gen_raw[gen,:Bus_ID]] 
        push!(Generators,PowerSystems.ThermalDispatch(gen_raw[gen,1],true,bus_id[1],
                TechThermal(0,@NT(min=float(gen_raw[gen,12]),max=float(gen_raw[gen,11])),0,
                    @NT(min=float(gen_raw[gen,14]),max=float(gen_raw[gen,13])),
                    @NT(up=gen_raw[gen,:Ramp_Rate_MW_Min],down=gen_raw[gen,:Ramp_Rate_MW_Min]),
                    @NT(up=gen_raw[gen,:Min_Up_Time_Hr],down=gen_raw[gen,:Min_Down_Time_Hr])
                    ),
                EconThermal(gen_raw[gen,:PMax_MW],varcost,0.0,start_hr*fuel_cost,0.0,nothing)
                ))
    elseif gen_raw[gen,:Fuel] in ["Hydro"]
        bus_id = [bus[i] for i in 1:length(bus) if bus[i].number == gen_raw[gen,:Bus_ID]] 
        push!(Generators,HydroCurtailment(gen_raw[gen,:GEN_UID],true,bus_id[1],
                        TechHydro(float(gen_raw[gen,11]),0.0,@NT(min =float(gen_raw[gen,12]), max =float(gen_raw[gen,11])),0.0,
                            @NT(min=float(gen_raw[gen,14]),max=float(gen_raw[gen,13])),
                            @NT(up=gen_raw[gen,:Ramp_Rate_MW_Min],down=gen_raw[gen,:Ramp_Rate_MW_Min]),
                            @NT(up=gen_raw[gen,:Min_Up_Time_Hr],down=gen_raw[gen,:Min_Down_Time_Hr])),
                            0.0,
                            TimeSeries.TimeArray(DayAhead, HydroTS_raw[:,Symbol("x"*gen_raw[gen,:GEN_UID])])
                            ))
    elseif gen_raw[gen,:Fuel] in ["Solar","Wind"]
        bus_id = [bus[i] for i in 1:length(bus) if bus[i].number == gen_raw[gen,:Bus_ID]] 
        if gen_raw[gen,4] == "PV"
            ts_raw = TimeSeries.TimeArray(DayAhead, PVTS_raw[:,Symbol("x"*gen_raw[gen,:GEN_UID])])
        elseif  gen_raw[gen,4] == "RTPV"
            ts_raw = TimeSeries.TimeArray(DayAhead, RTPVTS_raw[:,Symbol("x"*gen_raw[gen,:GEN_UID])])
        elseif  gen_raw[gen,4] == "WIND"
            ts_raw = TimeSeries.TimeArray(DayAhead, WindTS_raw[:,Symbol("x"*gen_raw[gen,:GEN_UID])])
        elseif  gen_raw[gen,4] == "CSP"
            ts_raw = TimeSeries.TimeArray(DayAhead, CSPTS_raw[:,Symbol(gen_raw[gen,:GEN_UID])])
        end
#         info("Power Factor dont defined assumed to be .95")
        push!(Generators,RenewableCurtailment(gen_raw[gen,:GEN_UID],true,bus_id[1],
                            float(gen_raw[gen,12]),EconRenewable(0.0,nothing),ts_raw))
    end
end

In [46]:
Forecasts_dict =make_forecast_dict(HydroTS_raw,Day(1),24,Generators)

Dict{String,Any} with 1 entry:
  "Deterministic" => Dict{String,Any}(Pair{String,Any}("215_HYDRO_3", Dict{Stri…

In [48]:
Forecasts_dict["Deterministic"]["215_HYDRO_3"]["data"][DateTime(2020,02,13,00,00,00)][1]

25x1 TimeSeries.TimeArray{Union{Float64, Missings.Missing},1,DateTime,Array{Union{Float64, Missings.Missing},1}} 2020-02-13T00:00:00 to 2020-02-14T00:00:00
│                     │       │
├─────────────────────┼───────┤
│ 2020-02-13T00:00:00 │ 26.9  │
│ 2020-02-13T01:00:00 │ 14.9  │
│ 2020-02-13T02:00:00 │ 18.5  │
│ 2020-02-13T03:00:00 │ 13.9  │
│ 2020-02-13T04:00:00 │ 14.2  │
│ 2020-02-13T05:00:00 │ 14.1  │
│ 2020-02-13T06:00:00 │ 18.7  │
│ 2020-02-13T07:00:00 │ 23.8  │
│ 2020-02-13T08:00:00 │ 24.4  │
│ 2020-02-13T09:00:00 │ 20.8  │
│ 2020-02-13T10:00:00 │ 25.3  │
│ 2020-02-13T11:00:00 │ 19.5  │
   ⋮
│ 2020-02-13T14:00:00 │ 10.1  │
│ 2020-02-13T15:00:00 │ 8.9   │
│ 2020-02-13T16:00:00 │ 6.9   │
│ 2020-02-13T17:00:00 │ 7.1   │
│ 2020-02-13T18:00:00 │ 13.4  │
│ 2020-02-13T19:00:00 │ 15.4  │
│ 2020-02-13T20:00:00 │ 9.6   │
│ 2020-02-13T21:00:00 │ 13.4  │
│ 2020-02-13T22:00:00 │ 34.8  │
│ 2020-02-13T23:00:00 │ 10.6  │
│ 2020-02-14T00:00:00 │ 7.7   │

In [49]:
for (type_key,type_fc) in Forecasts_dict
    for (key,det_dicts) in type_fc
        stringdata =JSON.json(det_dicts, 3)
        open("JSON/$key.json", "w") do f
            write(f, stringdata)
         end
    end
end

In [67]:
dict2 =Dict()
open("JSON/122_HYDRO_1.json", "r") do f
    global dict2
    dicttxt = readstring(f)  # file information to string
    dict2=JSON.parse(dicttxt)  # parse and transform data
end

Dict{String,Any} with 6 entries:
  "initialtime" => "2020-01-01T00:00:00"
  "resolution"  => Dict{String,Any}(Pair{String,Any}("value", 1))
  "interval"    => Dict{String,Any}(Pair{String,Any}("value", 60))
  "device"      => Dict{String,Any}(Pair{String,Any}("name", "122_HYDRO_1"),Pai…
  "horizon"     => 24
  "data"        => Dict{String,Any}(Pair{String,Any}("2020-05-07T00:00:00", Dic…

In [69]:
Array{PowerSystems.PowerSystemDevice}(Generators)

154-element Array{PowerSystems.PowerSystemDevice,1}:
 PowerSystems.ThermalDispatch(name="101_CT_1")        
 PowerSystems.ThermalDispatch(name="101_CT_2")        
 PowerSystems.ThermalDispatch(name="101_STEAM_3")     
 PowerSystems.ThermalDispatch(name="101_STEAM_4")     
 PowerSystems.ThermalDispatch(name="102_CT_1")        
 PowerSystems.ThermalDispatch(name="102_CT_2")        
 PowerSystems.ThermalDispatch(name="102_STEAM_3")     
 PowerSystems.ThermalDispatch(name="102_STEAM_4")     
 PowerSystems.ThermalDispatch(name="107_CC_1")        
 PowerSystems.ThermalDispatch(name="113_CT_1")        
 PowerSystems.ThermalDispatch(name="113_CT_2")        
 PowerSystems.ThermalDispatch(name="113_CT_3")        
 PowerSystems.ThermalDispatch(name="113_CT_4")        
 ⋮                                                    
 PowerSystems.RenewableCurtailment(name="118_RTPV_5") 
 PowerSystems.RenewableCurtailment(name="118_RTPV_6") 
 PowerSystems.RenewableCurtailment(name="320_RTPV_6") 
 PowerSystem

In [63]:
macro string_as_varname_macro(s::Any, v::Any)
    s = Symbol(s)
    esc(:($s = $v))
end

@string_as_varname_macro (macro with 2 methods)

In [59]:
device_names = [d.name for d in Generators];

In [64]:
@string_as_varname_macro(device_names[1],Dict())

Dict{Any,Any} with 0 entries

In [62]:
device_names[1]

"101_CT_1"

LoadError: [91mUndefVarError: _CT_1 not defined[39m